In [1]:
import sys
sys.path.append('../')

import networkx as nx
from node2vec import Node2Vec

from src.utils import load_embeddings

%load_ext autoreload
%autoreload 2

In [2]:
G = nx.read_gpickle('data/network.gpickle')
G.number_of_nodes(), G.number_of_edges()

(233, 221)

In [3]:
def generate_node_embeddings(graph: nx.Graph):
    model = Node2Vec(graph, dimensions=32, num_walks=100, walk_length=80, weight_key='dist', temp_folder='temp/', workers=2)
    result = model.fit(window=5, min_count=1)
    return result

In [4]:
%%time
embeddings = generate_node_embeddings(G)

Generating walks (CPU: 1): 100%|██████████| 50/50 [01:12<00:00,  1.45s/it]


CPU times: user 1min 20s, sys: 13.3 s, total: 1min 33s
Wall time: 1min 15s


In [5]:
for idx, e in enumerate(G.edges()):
    if idx == 5:
        break
    print(e)

('1 AV', '3 AV')
('1 AV', 'BEDFORD AV')
('103 ST', '96 ST')
('103 ST', 'CATHEDRAL PKWY')
('103 ST', '110 ST')


In [6]:
embeddings.wv.most_similar('1 AV')

[('BEDFORD AV', 0.8572643399238586),
 ('3 AV', 0.8287625312805176),
 ('LORIMER ST', 0.7262059450149536),
 ('GRAHAM AV', 0.676728367805481),
 ('HEWES ST', 0.6465608477592468),
 ('GRAND ST', 0.5279357433319092),
 ('MARCY AV', 0.5188640356063843),
 ('DEKALB AV', 0.4773113429546356),
 ('METROPOLITAN AV', 0.409771203994751),
 ('MONTROSE AV', 0.38509029150009155)]

In [7]:
embeddings.save('models_data/embeddings32.data')

In [ ]:
# join embeddings with corresponding nodes (as x feature)
for idx, node in enumerate(G.nodes()):
    G.add_node(node, embedding=embeddings.wv[str(node)].copy())

In [ ]:
nx.readwrite.write_gpickle(G, 'data/network32.gpickle')